<h1>Homework 04 - Applied ML</h1>

Importation of libraries

In [89]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier

Importation of the datas

In [104]:
filename = os.path.join('data','CrowdstormingDataJuly1st.csv') 
df = pd.read_csv(filename)
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


<h2>Data exploration</h2>

In [91]:
print('Number of dyads: ', len(df))
print('Total number of interactions between a referee and a player: ', sum(df.games))

Number of dyads:  146028
Total number of interactions between a referee and a player:  426572


In [105]:
print('Mean number of games for a dyad: ', np.mean(df['games']))

Mean number of games for a dyad:  2.921165803818446


## Data set up

In [106]:
print("We have", df.size,"rows.")

We have 4088784 rows.


We decided to remove row where the two rates were significantly different or if any of the rates were absent (Nan value).

In [121]:
cleandf = pd.DataFrame()

In [125]:
cleandf = df[df["rater1"].notnull() & df["rater2"].notnull()]
cleandf['mean'] = abs(cleandf.rater1 - cleandf.rater2)

cleandf = cleandf[cleandf['mean'] <= 0.25]
cleandf.drop('mean', axis =1, inplace=True)

In [127]:
cleandf.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [108]:
print("And now we have",cleandf.size,"rows")

And now we have 3614009 rows


In [109]:
print('Number of dyads: ', len(cleandf))
print('Total number of interactions between a referee and a player: ', sum(cleandf.games))

Number of dyads:  124621
Total number of interactions between a referee and a player:  373067


Then we decide to take the skin tone as the mean between the two rates

In [128]:
cleandf["meanSkinTone"] = abs(cleandf["rater1"] - cleandf["rater2"] ) / 2
cols = cleandf.columns.tolist()
cols = cols[-1:] + cols[:-1]
cleandf = cleandf[cols]
cleandf.head()

,meanSkinTone,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,0.125,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,0.000,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
5,0.125,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,0.125,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,0.000,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [129]:
def time_to_seconds(t):
    seconds = (pd.to_datetime(t) - datetime.datetime(1970, 1,1)).total_seconds()
    return int(seconds)

cleandf.birthday = cleandf.birthday.apply(time_to_seconds)

In [130]:
cleandf.head()

,meanSkinTone,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,0.125,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,431136000,177.0,72.0,Attacking Midfielder,1,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,0.000,john-utaka,John Utaka,Montpellier HSC,France,397008000,179.0,82.0,Right Winger,1,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
5,0.125,aaron-hughes,Aaron Hughes,Fulham FC,England,303177600,182.0,71.0,Center Back,1,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,0.125,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,497836800,187.0,80.0,Left Fullback,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,0.000,alexander-tettey,Alexander Tettey,Norwich City,England,512956800,180.0,68.0,Defensive Midfielder,1,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


Also, for our classifier to work, we remove all non numerical value by creatting dummy values to replace them.

In [ ]:
dummydf = pd.get_dummies(cleandf, prefix=None, prefix_sep='_', dummy_na=False, columns=["position"], sparse=False, drop_first=False)

In [ ]:
# count the number of club to check if it feasable to create dummy value for the feature "club"
print("The number of different clubs is",dummydf["club"].unique().size)

In [ ]:
dummydf = pd.get_dummies(dummydf, prefix=None, prefix_sep='_', dummy_na=False, columns=["club"], sparse=False, drop_first=False)
dummydf = pd.get_dummies(dummydf, prefix=None, prefix_sep='_', dummy_na=False, columns=["leagueCountry"], sparse=False, drop_first=False)

dummydf = dummydf.drop(['playerShort', 'player', 'photoID', 'rater1', 'rater2']  , axis=1)

In [ ]:
print("Now we have",dummydf.columns.size, "columns.")

In [ ]:
val = dummydf["Alpha_3"].drop_duplicates()
val.apply(lambda c :print(c))

## Classifier

In [ ]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [ ]:
def cross_validation(y, x, k_indices, k):
    """return the loss of ridge regression."""
    # get k'th subgroup in test, others in train
    te_indice = k_indices[k]
    tr_indice = k_indices[~(np.arange(k_indices.shape[0]) == k)]
    tr_indice = tr_indice.reshape(-1)
    y_te = y[te_indice]
    y_tr = y[tr_indice]
    x_te = x[te_indice]
    x_tr = x[tr_indice]
    
    #Make classifier
    rf = RandomForestClassifier(n_jobs = 4) #TODO: *maybe* add parameters

    #Train the classifier
    rf.fit(x_tr, y_tr)

    # Make prediction for testing data with the classifier
    y_pred = clf.predict(x_te)

    #Compute error (trying with 2 different methods)
    rf_score1 = log_loss(y_te, y_pred)
    rf_score2 = metrics.accuracy_score(y_te, y_pred)

    print('score: ', rf_score1)
    print('score 2: ', rf_score2)
    

In [ ]:
array = pd.DataFrame.as_matrix(dummydf)
array[:,1:]

In [ ]:
k_fold = 4
y = array[:, 0]
x = array[:, 1:]
k_indices = build_k_indices(y, k_fold, 1)
for k in range(k_fold):
    cross_validation(y, x, k_indices, k)

In [ ]:
X = df.drop(['rater1', 'rater2'], axis=1)
y = df.rater1

# TODO: Change that with real cross validation
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#Make classifier
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier() #TODO: *maybe* add parameters

#Train the classifier
rf.fit(X_train, y_train)

# Make prediction for testing data with the classifier
y_pred = clf.predict(X_test)

#Compute error (trying with 2 different methods)
rf_score1 = log_loss(y_test, y_pred)
rf_score2 = metrics.accuracy_score(y_test, y_pred)

print('score: ', rf_score1)
print('score 2: ', rf_score2)